Based on https://huggingface.co/docs/transformers/tasks/sequence_classification

In [2]:
import pandas
reviews = pandas.read_csv("data/reviews.csv")
reviews.head()

,review_id,vehicle_id,email,screen_name,review_date,status,global_review,create_date,last_changed_date,global_rating,global_title,updated_by
0,953285,50215,JOSEANT1710@GMAIL.COM,Jose,2021-01-01 00:51:25,5,I only buy toyota they don't give any mechanic...,2021-01-01 00:51:26,2021-02-20 11:04:33,5,Great love it,Monica Davis
1,953295,8212,HOLLISCROSS621@YAHOO.COM,Subaru Fan,2021-01-01 00:53:36,5,Been the sporty 2nd car as we have had 2 cars ...,2021-01-01 00:53:36,2021-02-12 10:26:29,5,Hardly been driven,Caden Smith
2,953305,53775,KEYELEMENTS8@GMAIL.COM,WDS,2021-01-01 01:03:37,3,These are the features I like on this vehicle....,2021-01-01 01:03:37,2021-03-09 20:24:32,5,Features,Caden Smith
3,953405,51833,MANTSBLOG@GMAIL.COM,Nino,2021-01-01 02:24:36,3,"Great, fun, energetic car! I'm 6'2"" and have a...",2021-01-01 02:24:36,2021-03-12 15:20:54,5,Great car!,Aidan Bretzel
4,953415,54993,SNEED.BRODERICK@YAHOO.COM,BMW650IGranCoupe,2021-01-01 03:12:19,3,This car is hAs so many safety features. Blin...,2021-01-01 03:12:18,2021-02-26 10:58:13,5,True performance,Bill Smith


In [3]:
reviews['label'] = reviews.apply(lambda row: 1 if row['status'] == 3.0 else 0, axis=1)



In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [5]:

def preprocess_function(row):
    return tokenizer(str(row['global_review']), truncation=True)


In [6]:
train = reviews.sample(frac=0.8, random_state= 25)
test = reviews.drop(train.index)

from datasets import Dataset
train_ds = Dataset.from_pandas(train[['global_review','label']])
test_ds = Dataset.from_pandas(test[['global_review','label']])

tokenized_train_ds = train_ds.map(preprocess_function)
tokenized_test_ds = test_ds.map(preprocess_function)

  0%|          | 0/228343 [00:00<?, ?ex/s]

  0%|          | 0/57086 [00:00<?, ?ex/s]

In [7]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
##model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 2)

Loading model from a savepoint (in case training gets interrupted)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("results/checkpoint-35000")

Getting a prediction on a single review. First number is likelihood for rejection, second is likelihood for acceptance. Negative means low likelihood.

In [15]:
inputs = tokenizer("This is a good car I drive it all day ass dammit.", return_tensors="pt")
outputs = model(**inputs)
print (outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.4363, -1.2542]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [66]:
training_args = TrainingArguments(
    output_dir="./results2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_test_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [1]:
trainer.train()


NameError: name 'trainer' is not defined